# Tasca M10 T01

**Exercicis de Web Scraping.**

## Exercici 1
**Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després.**

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)


El web scraping es una técnica utilizada para extraer información y datos de sitios web de manera automatizada. Consiste en procesar el contenido de una página web, ya sea HTML, XML, JSON u otros formatos, para obtener datos específicos de interés.

Esta técnica se emplea para diversas finalidades, como:

- Recopilación de Datos: Obtener información de productos, precios, opiniones, noticias, y cualquier otro tipo de datos disponibles en sitios web públicos.

- Análisis y Monitorización: Recopilar datos para análisis de mercado, seguimiento de tendencias, vigilancia competitiva o monitoreo de cambios en los sitios web.

- Integración de Datos: Extraer datos de múltiples fuentes web para integrarlos en una base de datos o sistema propio.

El web scraping se puede realizar de diferentes maneras:

- Parsing (Análisis): Analizar el código HTML o XML de una página web para extraer datos utilizando bibliotecas como Beautiful Soup o lxml en Python.

- Automatización de Navegadores: Utilizar herramientas como Selenium para automatizar la interacción con páginas web a través de un navegador, permitiendo la extracción de datos de sitios web dinámicos o que requieren interacciones complejas.

- Frameworks Específicos: Emplear frameworks dedicados como Scrapy en Python, que permiten crear arañas (spiders) para recorrer sitios web, extraer datos estructurados y gestionar el scraping a gran escala.

A lo largo de esta tarea, examinaremos un ejemplo de cada uno de estos métodos, brindando una primera impresión para evaluar en el futuro cuál de ellos se ajusta mejor a nuestras necesidades. Para iniciar, en el primer ejercicio se requiere realizar web scraping con Beautiful Soup y Selenium en dos páginas web. La primera página será un anexo de la Wikipedia, desde donde se extraerá una tabla del medallero olímpico utilizando Beautiful Soup como punto de partida.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos'
wiki = requests.get(url)
soup = BeautifulSoup(wiki.text, "html.parser")
table = soup.find('tbody')
filas = table.find_all('tr')[1:]

data = []

for fila in filas:
    columnas = fila.find_all('td')
    datos_fila = [columna.text for columna in columnas]
    data.append(datos_fila)

nombres = ['PAIS','NJV','OJV','PJV','BJV','TJV','NJI','OJI','PJI','BJI','TJI','NJO','OJO','PJO','BJO','TJO']
df = pd.DataFrame(data,columns= nombres)
df

,PAIS,NJV,OJV,PJV,BJV,TJV,NJI,OJI,PJI,BJI,TJI,NJO,OJO,PJO,BJO,TJO
0,Afganistán (AFG)\n,15,0,0,2,2\n,0,0,0,0,0\n,15\n,0,0,2,2\n
1,Alemania (GER)[3]​[4]​\n,17,239,267,291,797\n,12,112,104,70,286\n,29,351,371,361,1083\n
2,Alemania Occidental (FRG)[3]​\n,5,56,67,81,204\n,7,11,15,13,39\n,12,67,82,94,243\n
3,Alemania Oriental (GDR)[3]​\n,5,153,129,127,409\n,6,39,36,35,110\n,11,192,165,162,519\n
4,Antillas Neerlandesas (AHO)\n,13,0,1,0,1\n,2,0,0,0,0\n,15,0,1,0,1\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Vietnam (VIE)\n,16,1,3,1,5\n,0,0,0,0,0\n,16,1,3,1,5\n
149,Yibuti (DJI)\n,8,0,0,1,1\n,0,0,0,0,0\n,8,0,0,1,1\n
150,Yugoslavia (YUG)[12]​\n,18,26,29,28,83\n,16,0,3,1,4\n,34,26,32,29,87\n
151,Zambia (ZAM)\n,14,0,1,1,2\n,0,0,0,0,0\n,14,0,1,1,2\n


Observamos que con un código bastante reducido logramos extraer toda una tabla de dimensiones 153 filas por 16 columnas en muy poco tiempo. Sin embargo, para trabajar con estos datos, necesitaríamos realizar un análisis adicional debido a que algunas celdas contienen el carácter '\n', lo cual parece ser un salto de línea. Además, fue necesario comenzar desde el segundo 'tr' al raspar el código, ya que aunque en la estructura original de la página web se diferenciaban entre '/tbody' y '/thead', al extraer el código, al menos con el analizador "html.parser", no se encontraba la etiqueta '/thead' y en su lugar se modificaba por '/tbody'.

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

options = Options()
options.add_argument('--headless')
options.add_argument('--incognito')

url_wiki = 'https://es.wikipedia.org/wiki/Anexo:Medallero_de_los_Juegos_Ol%C3%ADmpicos'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(url_wiki)
time.sleep(5) 

filas = driver.find_elements(By.XPATH, '//center/table/tbody/tr')

data = []

for fila in filas:
    columnas = fila.find_elements(By.TAG_NAME, 'td')  
    datos_fila = [columna.text for columna in columnas]  
    data.append(datos_fila)

driver.quit()

nombres = ['PAIS','NJV','OJV','PJV','BJV','TJV','NJI','OJI','PJI','BJI','TJI','NJO','OJO','PJO','BJO','TJO']
df = pd.DataFrame(data,columns= nombres)
df

,PAIS,NJV,OJV,PJV,BJV,TJV,NJI,OJI,PJI,BJI,TJI,NJO,OJO,PJO,BJO,TJO
0,Afganistán (AFG),15,0,0,2,2,0,0,0,0,0,15,0,0,2,2
1,Alemania (GER)34,17,239,267,291,797,12,112,104,70,286,29,351,371,361,1083
2,Alemania Occidental (FRG)3,5,56,67,81,204,7,11,15,13,39,12,67,82,94,243
3,Alemania Oriental (GDR)3,5,153,129,127,409,6,39,36,35,110,11,192,165,162,519
4,Antillas Neerlandesas (AHO),13,0,1,0,1,2,0,0,0,0,15,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Vietnam (VIE),16,1,3,1,5,0,0,0,0,0,16,1,3,1,5
149,Yibuti (DJI),8,0,0,1,1,0,0,0,0,0,8,0,0,1,1
150,Yugoslavia (YUG)12,18,26,29,28,83,16,0,3,1,4,34,26,32,29,87
151,Zambia (ZAM),14,0,1,1,2,0,0,0,0,0,14,0,1,1,2


La primera diferencia notable reside en los tiempos de procesamiento. A pesar de eliminar la función "time.sleep", notamos una diferencia significativa en la velocidad entre Beautiful Soup y Selenium. Esto podría atribuirse a la mayor potencia y la ejecución de múltiples procesos en Selenium, lo que lleva a un rendimiento más lento en comparación con Beautiful Soup. Además, la configuración necesaria para utilizar Selenium es un poco más extensa, pero a cambio, proporciona resultados considerablemente más limpios.

La primera página no presentó dificultades. Sin embargo, en el segundo ejemplo, abordamos la Bolsa de Madrid. Aquí, la situación cambia: se trata de una página dinámica donde los valores de las acciones se actualizan en tiempo real mediante JavaScript. Como mencionamos al principio, Beautiful Soup analiza código HTML o XML; por lo tanto, al intentar raspar la página, encontramos que partes del código no son visibles, lo que hace imposible extraer la tabla de valores del IBEX35. Para resolver esto, recurro a Selenium, que sí funciona con páginas web dinámicas para llevar a cabo el scraping. Una vez obtenidos los datos, empleo Beautiful Soup para extraer tanto el encabezado (que corresponde a los nombres de las columnas de la tabla) como la tabla en sí.

In [3]:
url_bolsa = 'https://www.bolsasymercados.es/bme-exchange/es/Mercados-y-Cotizaciones/Acciones/Mercado-Continuo/Precios/ibex-35-ES0SI0000005'
driver = webdriver.Chrome(options=options)
driver.get(url_bolsa)
time.sleep(5)
html = driver.page_source
soup_bolsa = BeautifulSoup(html, "html.parser")
results = soup_bolsa.find('table', {'role':'table','class':'shares-table'})

columnas = results.find('thead').find_all('th')
textos_columnas = [columna.text for columna in columnas]

filas = results.find('tbody').find_all('tr')
data = []

for fila in filas:
    columnas = fila.find_all('td')
    datos_fila = [columna.text for columna in columnas]
    data.append(datos_fila)

df = pd.DataFrame(data, columns=textos_columnas)
df

,Nombre,Último,% Dif.,Máximo,Mínimo,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"131,2000","0,96%","131,4500","130,1500",13.204,"1.728,46",13/12/2023,12:17:26
1,ACCIONA ENER,"26,3200","0,61%","26,4600","26,1200",44.258,"1.164,49",13/12/2023,12:16:45
2,ACERINOX,"10,0300","0,00%","10,1050","9,9920",289.365,"2.909,20",13/12/2023,12:18:09
3,ACS,"38,7600","2,05%","38,8000","38,0800",217.251,"8.364,46",13/12/2023,12:17:56
4,AENA,"162,6000","0,90%","162,9000","161,2500",18.422,"2.988,20",13/12/2023,12:17:27
5,AMADEUS,"65,6200","0,03%","66,0200","65,5200",62.018,"4.080,11",13/12/2023,12:16:35
6,ARCELORMIT.,"24,1500","0,75%","24,2150","23,9600",47.275,"1.141,06",13/12/2023,12:15:34
7,B.SANTANDER,"3,8455","-0,77%","3,8800","3,8220",9.479.252,"36.511,81",13/12/2023,12:18:00
8,BA.SABADELL,"1,2235","-0,20%","1,2350","1,2170",5.416.953,"6.637,05",13/12/2023,12:18:00
9,BANKINTER,"6,0620","0,87%","6,0720","5,9760",871.896,"5.260,98",13/12/2023,12:18:00


Una vez más, podemos apreciar la eficacia de la biblioteca Beautiful Soup. Hemos logrado extraer rápidamente una tabla de 35 filas y 9 columnas junto con su encabezado correspondiente con muy pocas líneas de código. En esta ocasión, ninguna celda necesita limpieza, probablemente debido a que el raspado se realizó con Selenium. Aprovechando esta situación, procederemos a extraer la tabla pero esta vez utilizando Selenium a partir de la variable "driver" ya obtenida anteriormente.

In [4]:
columnas = driver.find_elements(By.XPATH, '//table[@class="shares-table"]/thead/tr/th')
textos_columnas = [columna.text for columna in columnas]

filas = driver.find_elements(By.XPATH, '//table[@class="shares-table"]/tbody[@role="rowgroup"]/tr')

data = []

for fila in filas:
    columnas = fila.find_elements(By.TAG_NAME, 'td')  
    datos_fila = [columna.text for columna in columnas]  
    data.append(datos_fila)

driver.quit()

df = pd.DataFrame(data,columns=textos_columnas)
df

,Nombre,Último,% Dif.,Máximo,Mínimo,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"131,2000","0,96%","131,4500","130,1500",13.204,"1.728,46",13/12/2023,12:17:26
1,ACCIONA ENER,"26,3200","0,61%","26,4600","26,1200",44.258,"1.164,49",13/12/2023,12:16:45
2,ACERINOX,"10,0300","0,00%","10,1050","9,9920",289.365,"2.909,20",13/12/2023,12:18:09
3,ACS,"38,7600","2,05%","38,8000","38,0800",217.251,"8.364,46",13/12/2023,12:17:56
4,AENA,"162,6000","0,90%","162,9000","161,2500",18.422,"2.988,20",13/12/2023,12:17:27
5,AMADEUS,"65,6200","0,03%","66,0200","65,5200",62.018,"4.080,11",13/12/2023,12:16:35
6,ARCELORMIT.,"24,1500","0,75%","24,2150","23,9600",47.275,"1.141,06",13/12/2023,12:15:34
7,B.SANTANDER,"3,8455","-0,77%","3,8800","3,8220",9.479.252,"36.511,81",13/12/2023,12:18:00
8,BA.SABADELL,"1,2235","-0,20%","1,2350","1,2170",5.416.953,"6.637,05",13/12/2023,12:18:00
9,BANKINTER,"6,0620","0,87%","6,0720","5,9760",871.896,"5.260,98",13/12/2023,12:18:00


Una vez obtenido el raspado que es la parte mas lenta, extraer la tabla se ejecuta en un momento y con un código,que aunque diferente al de Beautiful Soup, de una misma complejidad.

En conclusión, Selenium y Beautiful Soup son dos herramientas diferentes utilizadas en el ámbito del web scraping, pero tienen propósitos y funcionalidades distintas:

**Selenium**:

- Propósito: Es una herramienta diseñada para la automatización de navegadores web. Puede controlar un navegador web real (Chrome, Firefox, etc.) de manera programática, permitiendo interactuar con el contenido de una página web, hacer clic en botones, rellenar formularios, y más.
- Uso principal: Útil para casos en los que se requiere interactuar con una página web como lo haría un usuario real. Es ideal para aplicaciones web que utilizan JavaScript u operan con contenido dinámico cargado por AJAX.

**Beautiful Soup**:

- Propósito: Es una biblioteca de Python utilizada para analizar y extraer datos de páginas web. Se utiliza para analizar la estructura HTML/XML de una página y extraer la información deseada.
- Uso principal: Ideal para extraer datos estáticos de una página web, como raspar información de tablas, listas, encabezados, párrafos, etc.

En resumen, Selenium se utiliza principalmente para interactuar con páginas web y simular la interacción humana con un navegador, mientras que Beautiful Soup se centra en analizar la estructura HTML de una página para extraer datos específicos sin interactuar con ella directamente. A menudo, ambas herramientas se utilizan en conjunto: Selenium para interactuar con la página y Beautiful Soup para analizar y extraer la información necesaria una vez que se ha cargado.

## Exercici 2
**Documenta en un Word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.**

A manera d'exemple del que es demana pots consultar aquest enllaç:

->https://www.kaggle.com/datasets/vivovinco/20212022-football-team-stats.

## Exercici 3
**Tria una pàgina web que tu vulguis i realitza web scraping mitjançant la llibreria Selenium primer i Scrapy després.**

Scrapy es un framework de scraping de código abierto basado en Python, y se diferencia de Selenium y Beautiful Soup en varios aspectos:

- Enfoque y Funcionalidad: Está diseñado específicamente para realizar web scraping a gran escala y para la creación de arañas (spiders) que puedan extraer datos de manera estructurada y sistemática de múltiples páginas web de manera eficiente.
- Funcionalidades Adicionales: Ofrece características adicionales como manejo de solicitudes y respuestas HTTP, seguimiento de enlaces, manejo de sesiones, pipelines para procesar datos extraídos, gestión de proxies, y más, lo que lo hace ideal para proyectos de web scraping a gran escala.
- Escala y Complejidad: Es más adecuado para proyectos de scraping complejos y a gran escala, donde se necesita rastrear múltiples sitios web, seguir enlaces, y extraer grandes volúmenes de datos estructurados.

Para adentrarnos en el mundo de Scrapy he elegido la pagina web de filmaffinity de donde intentaré extraer la tabla con las peliculas mejor valoradas.

In [5]:
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess

class FilmaffinitySpider(scrapy.Spider):
    name = 'filmaffinity'
    start_urls = ['https://www.filmaffinity.com/es/topgen.php?genre=&fromyear=&toyear=&country=&nodoc']

    def parse(self, response):
        for movie in response.xpath('//ul[@id="top-movies"]/li/ul'):
            position = movie.xpath('.//li[@class="position"]/text()').get()
            title = movie.xpath('.//div[@class="mc-title"]/a/text()').get()
            directors = movie.xpath('.//span[@class="nb"]/a/text()').getall()
            rating = movie.xpath('.//li[@class="data"]/div[@class="avg-rating"]/text()').get()

            if directors:
                all_directors = ', '.join(directors)
            
            yield {
                'Position' : position,
                'Title': title,
                'Director': all_directors,
                'Rating': rating
            }

process = CrawlerProcess(settings={
    'FEED_FORMAT': 'csv',
    'FEED_URI': 'top10_filmaffinity.csv',
    'LOG_LEVEL': 'ERROR',
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
})
process.crawl(FilmaffinitySpider)
process.start()

df = pd.read_csv('top10_filmaffinity.csv')
df

,Position,Title,Director,Rating
0,1,El padrino,"Francis Ford Coppola, Marlon Brando, Al Pacino...","9,0"
1,2,Planeta Tierra II (Miniserie de TV),"Elizabeth White, Justin Anderson, Ed Charles, ...","8,9"
2,3,El padrino. Parte II,"Francis Ford Coppola, Al Pacino, Robert De Nir...","8,9"
3,4,The Wire (Bajo escucha) (Serie de TV),"David Simon, Joe Chappelle, Ernest R. Dickerso...","8,8"
4,5,Breaking Bad (Serie de TV),"Vince Gilligan, Michelle MacLaren, Adam Bernst...","8,8"
...,...,...,...,...
583,26,Hermanos de sangre (Miniserie de TV),"Stephen Ambrose, David Frankel, Mikael Salomon...","8,5"
584,27,Harakiri,"Masaki Kobayashi, Tatsuya Nakadai, Rentarô Mik...","8,5"
585,28,El crepúsculo de los dioses,"Billy Wilder, William Holden, Gloria Swanson, ...","8,5"
586,29,Nuestro planeta (Serie de TV),"David Attenborough, Adam Chapman, Hugh Pearson...","8,5"


Vemos que los resultados son excelentes, tiempos de carga han sido muy rápidos aunque es cierto que la carga de información no era tan grande como los anteriores. El código es mas complejo, requiere la creacción de "arañas" que llevaran toda la información de la busqueda. Para el propósito que nos ocupa, puede parecer demasiado complejo para obtener una única tabla de una página estática, pero resulta ser una herramienta muy potente para escenarios que implican múltiples exploraciones en distintas páginas web. En resumen, Scrapy se muestra como un framework más completo y orientado a proyectos de web scraping de mayor complejidad. 

En conclusión, el web scraping ha experimentado un crecimiento significativo en los últimos años gracias a su capacidad para recopilar información valiosa de sitios web de manera automatizada. Esta técnica permite extraer datos estructurados de la web de una manera más rápida y eficiente que la recopilación manual, lo que ha llevado a su popularización en diversas industrias, desde la investigación académica hasta el análisis de mercado.

Sin embargo, a pesar de sus beneficios, el web scraping plantea una serie de preocupaciones éticas, legales y técnicas que lo convierten en un tema delicado y controvertido por varias razones:

- Cumplimiento Legal: El scraping de datos de sitios web puede estar sujeto a leyes y regulaciones. Algunos países tienen leyes que regulan la recopilación de datos, como la Ley de Portabilidad y Responsabilidad de Seguros de Salud (HIPAA) en EE. UU. o la Regulación General de Protección de Datos (GDPR) en la Unión Europea. Incumplir estas leyes puede resultar en acciones legales.

- Respeto por los Términos de Servicio: Muchos sitios web tienen términos de servicio que prohíben o restringen la recopilación automatizada de datos (scraping). Ignorar estos términos podría resultar en acciones legales por parte de los propietarios del sitio, vulnerando sus derechos de autor.

- Sobrecarga del Servidor: El scraping intensivo puede generar una carga significativa en los servidores del sitio web objetivo, lo que puede resultar en un rendimiento deficiente o la caída del sitio. Esto se considera un uso no ético y puede afectar negativamente a otros usuarios.

- Protección de la Privacidad: La recopilación de datos de sitios web puede implicar la extracción de información personal o sensible. Es importante ser cuidadoso y respetar la privacidad de los usuarios al manipular y utilizar estos datos.

- Preservar la Reputación: El scraping irresponsable o no ético puede dañar la reputación de la persona o empresa que realiza estas acciones. El respeto a las políticas y la ética refuerza la reputación y la confianza en el uso de tecnologías de extracción de datos.

- La calidad de los datos obtenidos: Los sitios web pueden cambiar su estructura, diseño o contenido sin previo aviso, lo que hace que los scripts de scraping pierdan su eficacia y precisión.

En resumen, aunque el web scraping ofrece ventajas innegables en términos de acceso a datos y automatización de procesos, su uso plantea desafíos éticos, legales y técnicos que requieren un enfoque cuidadoso y responsable por parte de quienes lo emplean. Es esencial comprender y respetar las políticas de uso de los sitios web y considerar las implicaciones legales y éticas antes de realizar actividades de web scraping.

Una solución a estos inconvenientes es el uso de APIs, la elección entre hacer scraping y utilizar una API (Interfaz de Programación de Aplicaciones) depende de varios factores:

- Disponibilidad de datos: Las APIs suelen ofrecer datos estructurados y específicos, mientras que el scraping puede extraer datos más detallados o específicos que pueden no estar disponibles a través de una API.

- Acceso y fiabilidad: Las APIs son más fiables y estables, ya que están diseñadas para proporcionar datos de manera estructurada y consistente. El scraping puede ser menos confiable ya que depende de la estructura y cambios del sitio web.

- Velocidad y rendimiento: Las APIs pueden ser más rápidas y eficientes para obtener datos, ya que a menudo permiten solicitudes en lotes y tienen límites de acceso bien definidos. El scraping puede ser más lento debido a la necesidad de navegar por páginas web y extraer datos.

- Legalidad y permisos: Algunos sitios web prohíben el scraping en sus términos de servicio, mientras que otros pueden ofrecer APIs para acceder a sus datos de manera legítima. Es importante respetar las políticas del sitio web al realizar scraping.

- Complejidad técnica: La implementación de scraping puede ser más compleja y requiere conocimientos avanzados de HTML, CSS, JavaScript y técnicas de extracción de datos. Las APIs suelen tener una documentación clara y son más fáciles de usar.

En general, si una API proporciona los datos que necesitas y tiene un límite adecuado para tus necesidades, es preferible usar la API debido a su fiabilidad y accesibilidad. Sin embargo, si los datos no están disponibles a través de una API o el scraping es la única opción viable, es crucial hacerlo de manera ética y respetando las políticas del sitio web.